# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
#corrected-data_path = 'weatherAPI_data.csv'
corrected_data_path = "Resources/cities.csv"

corrected_data_df = pd.read_csv(corrected_data_path)
corrected_data_df


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05
...,...,...,...,...,...,...,...,...,...,...
543,543,yabrud,0,SY,1558378840,32,33.97,36.66,80.60,8.05
544,544,paraiso,5,MX,1558378898,4,24.01,-104.61,84.20,16.11
545,545,veraval,0,FR,1558378898,62,49.65,0.71,61.00,8.05
546,546,novyy urgal,100,RU,1558378899,93,51.07,132.56,49.74,2.68


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
gmaps.configure(api_key=g_key)
locations = corrected_data_df[["Lat","Lng"]]
humidity = corrected_data_df["Humidity"]

In [24]:
fig = gmaps.figure(center=(50.0,-5.0), zoom_level=1)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)
# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [34]:
#humidity below 60 and wind speed below 5  this is not working??????
new_df = corrected_data_df[(corrected_data_df["Cloudiness"] == 0) & (corrected_data_df["Wind Speed"] <= 5) & (corrected_data_df["Humidity"] <= 60)].dropna()
new_df


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
31,31,qingdao,0,CN,1558378760,54,36.09,120.35,59.00,2.24
142,142,sarai mir,0,IN,1558378781,35,26.02,82.92,86.00,3.36
271,271,coihaique,0,CL,1558378837,59,-45.58,-72.07,46.40,4.70
297,297,mogok,0,MM,1558378844,51,22.92,96.51,72.24,2.77
346,346,chiredzi,0,ZW,1558378856,42,-21.05,31.67,65.76,0.78
500,500,mandalgovi,0,MN,1558378890,36,45.76,106.27,48.66,1.34


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [44]:
hotel_df = new_df.loc[:,["City_ID", "City", "Country", "Lat", "Lng"]]
hotel_df
hotel_df["Hotel_Name"]=""
hotel_df

,City_ID,City,Country,Lat,Lng,Hotel_Name
0,0,ostrovnoy,RU,68.05,39.51,
1,1,mahebourg,MU,-20.41,57.70,
2,2,qaanaaq,GL,77.48,-69.36,
3,3,zhuhai,CN,40.71,112.04,
4,4,cape town,ZA,-33.93,18.42,
...,...,...,...,...,...,...
543,543,yabrud,SY,33.97,36.66,
544,544,paraiso,MX,24.01,-104.61,
545,545,veraval,FR,49.65,0.71,
546,546,novyy urgal,RU,51.07,132.56,


In [50]:
# find the closest restaurant of each type to coordinates

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "radius": 5000 , 
    "keyword": "hotel",
    "type": "hotel",
    "key": g_key,
}
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    time.sleep(1)    
    # get hotel location
    lat = row['Lat']
    lng = row["Lng"]
    city_name = row["City"]
    
    # add keyword to params dict
    params['type'] = "lodging"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        
        types_df.loc[index, 'name'] = results[0]['name']
        #types_df.loc[index, 'address'] = results[0]['vicinity']
        #types_df.loc[index, 'price_level'] = results[0]['price_level']
        #types_df.loc[index, 'rating'] = results[0]['rating']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 0: ostrovnoy.
Missing field/result... skipping.
------------
Retrieving Results for Index 1: mahebourg.
Missing field/result... skipping.
------------
Retrieving Results for Index 2: qaanaaq.
Missing field/result... skipping.
------------
Retrieving Results for Index 3: zhuhai.
Missing field/result... skipping.
------------
Retrieving Results for Index 4: cape town.
Missing field/result... skipping.
------------
Retrieving Results for Index 5: atuona.
Missing field/result... skipping.
------------
Retrieving Results for Index 6: flinders.
Missing field/result... skipping.
------------
Retrieving Results for Index 7: opuwo.
Missing field/result... skipping.
------------
Retrieving Results for Index 8: tiksi.
Missing field/result... skipping.
------------
Retrieving Results for Index 9: pushkinskiye gory.
Missing field/result... skipping.
------------
Retrieving Results for Index 10: aykhal.
Missing field/result... skipping.
------------
Retrieving Results fo

Missing field/result... skipping.
------------
Retrieving Results for Index 93: antioch.
Missing field/result... skipping.
------------
Retrieving Results for Index 94: najran.
Missing field/result... skipping.
------------
Retrieving Results for Index 95: agirish.
Missing field/result... skipping.
------------
Retrieving Results for Index 96: mataura.
Missing field/result... skipping.
------------
Retrieving Results for Index 97: katsuura.
Missing field/result... skipping.
------------
Retrieving Results for Index 98: shalinskoye.
Missing field/result... skipping.
------------
Retrieving Results for Index 99: barrow.
Missing field/result... skipping.
------------
Retrieving Results for Index 100: sorong.
Missing field/result... skipping.
------------
Retrieving Results for Index 101: kingston.
Missing field/result... skipping.
------------
Retrieving Results for Index 102: ketchikan.
Missing field/result... skipping.
------------
Retrieving Results for Index 103: kapaa.
Missing field/

Missing field/result... skipping.
------------
Retrieving Results for Index 183: monrovia.
Missing field/result... skipping.
------------
Retrieving Results for Index 184: burnie.
Missing field/result... skipping.
------------
Retrieving Results for Index 185: kaitangata.


KeyboardInterrupt: 

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
